In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import root
import h5py
import os
import analysis

In [ ]:
# # Run to determine the difference in observables after gauge transformation.
# file = '/Users/theoares/lqcd/npr_momfrac/testing/output/cl3_16_48_b6p1_m0p2450_14503/cfg200.h5'
# f = h5py.File(file, 'r')
# propprime_p2222 = f['propprime/p2222/cfg200'][()]
# prop_p2222 = f['prop/p2222/cfg200'][()]
# delta = prop_p2222 - propprime_p2222
# scale = np.abs(prop_p2222[0, 0, 0, 0])
# print(np.max(delta / scale)) # first order entries are of size 10^4

In [ ]:
# file = '/Users/theoares/lqcd/npr_momfrac/examples/analysis_EXAMPLE_output.h5'
# f = h5py.File(file, 'r')
# S = f['prop/p2222'][()]

In [ ]:
# #confirm 2 point function is the same as Phiala's (it is)
# file = '/Users/theoares/lqcd/npr_momfrac/examples/meas/cl3_16_48_b6p1_m0p2450/pion_beam_npr_matrix_200.h5'
# f = h5py.File(file, 'r')
# S = f['prop/p2222'][()]

# file1 = '/Users/theoares/lqcd/npr_momfrac/output/cl3_16_48_b6p1_m0p2450_14570/cfg200.h5'
# f1 = h5py.File(file1, 'r')
# S1 = f['prop/p2222'][()]

# print(S - S1)

In [ ]:
tr1 = np.einsum('aiaj->ij', props['p2222'][0])
tr2 = np.einsum('aiaj->ij', props1['p2222'][0])
tr1[np.abs(tr1) < 10] = 0
tr2[np.abs(tr2) < 10] = 0
print(tr1)
print(tr2)

In [ ]:
cfgbase = 'cl3_16_48_b6p1_m0p2450'
# TODO when I do the next job, change how things are normalized
job_num = 14570
base = './output/' + cfgbase + '_' + str(job_num)

# base = './examples/analysis_EXAMPLE.py'
props, threepts = analysis.readfile(base)    # C should be a dictionary whose keys are the momentum slices.

V = analysis.get_hypervolume()
# for p in props:
#     props[p]= props[p] / V    # TODO keep this up to date with the QLUA normalization.

In [ ]:
print(Z['p2222'][0, 0])
print(Z1['p2222'][0, 0])

In [ ]:
mom_list = analysis.get_mom_list()
s = 0    # TODO seed this with something random
props_boot = analysis.bootstrap(props, seed = s)
threept_boot = analysis.bootstrap(threepts, seed = s)

In [ ]:
props_inv = analysis.invert_prop(props_boot)
Γ = analysis.amputate(props_inv, threept_boot)
Zq = analysis.quark_renorm(props_inv)
Γ_B, Γ_B_inv = analysis.born_term()
Z = analysis.get_Z(Zq, Γ, Γ_B_inv)

In [ ]:
μ, σ = analysis.get_statistics_Z(Z)
print(μ)
print(σ)

In [ ]:
p_squared = np.array([np.abs(analysis.square(p)) for p in mom_list])
μ_list = np.array([μ[analysis.plist_to_string(p)] for p in mom_list])
σ_list = np.array([σ[analysis.plist_to_string(p)] for p in mom_list])
plt.figure()
plt.errorbar(p_squared, np.abs(μ_list), yerr = np.abs(σ_list), fmt = 'o')
plt.ylabel('$\mathcal{Z}(p^2)$')
plt.xlabel('$-p^2$')
plt.title('Original field')
plt.show()

In [ ]:
props1, threepts1 = analysis.readfile(base, True)    # C should be a dictionary whose keys are the momentum slices.
# for p in props1:
#     props1[p]= props1[p] / V    # TODO keep this up to date with the QLUA normalization.

In [ ]:
props_boot1 = analysis.bootstrap(props1, seed = s)
threept_boot1 = analysis.bootstrap(threepts1, seed = s)

In [ ]:
props_inv1 = analysis.invert_prop(props_boot1)
Γ1 = analysis.amputate(props_inv1, threept_boot1)
Zq1 = analysis.quark_renorm(props_inv1)
Z1 = analysis.get_Z(Zq1, Γ1, Γ_B_inv)

In [ ]:
μ1, σ1 = analysis.get_statistics_Z(Z1)
print(μ1)
print(σ1)

In [ ]:
μ_list1 = np.array([μ1[analysis.plist_to_string(p)] for p in mom_list])
σ_list1 = np.array([σ1[analysis.plist_to_string(p)] for p in mom_list])
plt.figure()
plt.errorbar(p_squared, np.abs(μ_list1), yerr = np.abs(σ_list1), fmt = 'o')
plt.ylabel('$\mathcal{Z}_{gauge}(p^2)$')
plt.xlabel('$-p^2$')
plt.title('Gauge transformed field')
plt.show()

In [ ]:
Δ = np.abs(μ_list) - np.abs(μ_list1)
σ_Δ = np.sqrt(σ_list ** 2 + σ_list1 ** 2)
plt.figure()
plt.errorbar(p_squared, np.abs(Δ), yerr = np.abs(σ_Δ), fmt = 'o')
plt.ylabel('$(\mathcal{Z} - \mathcal{Z}_{gauge})(p^2)$')
plt.xlabel('$-p^2$')
plt.title('Difference in \mathcal{Z} values')
plt.show()

There is a phase difference of about -1 between the original field and the gauge transformed field. Is there something I'm doing that's systematically creating that after the code is run?